In [1]:
import pandas as pd 
import numpy as np 
import os as os
from datetime import datetime
import time
import json
import gzip
import pprint
import requests
import uuid

In [2]:
user_file = './sample_users_1000.csv'
item_file = './content_metadata.csv'

user_df = pd.read_csv(user_file)
item_df = pd.read_csv(item_file)

In [3]:
item_df.head()

,pushcall_id,base_tag,tag_2,tag_3,crop,month,duration
0,20210703_cucurbits_downy_mildew_dpm_crop,crop,pest management,downy_mildew,cucurbits,7,90.94
1,20210703_maize_faw_dpm_crop,crop,pest management,faw,maize,7,129.50
2,20210703_sugarcane_top_shoot_borer_dpm_crop,crop,pest management,top_shoot_borer,sugarcane,7,81.65
3,20210801_brinjal_shoot_and_fruit_borer_dpm_crop,crop,pest management,shoot_and_fruit_borer,brinjal,8,118.54
4,20210801_groundnut_leaf_miner_sucking_pest_dpm...,crop,pest management,leaf_miner_sucking_pest,groundnut,8,98.45


In [4]:
allowed_cols = ['base_tag','tag_2','crop']

In [5]:
# mapping an item with it's tag. Using base_tag tag_1 and crop to represent an id
idx_to_base_tag2_crop_map = {}
for _,row in item_df.iterrows():
    item_id = row['pushcall_id']
    val = None
    for col in allowed_cols:
        if val is None:
            val = row[col]
        else:
            val = f"{val}|{row[col]}"
    idx_to_base_tag2_crop_map[item_id] = val

In [6]:
# number of such permutation
print(len(idx_to_base_tag2_crop_map))
len(set(idx_to_base_tag2_crop_map.values()))

370


74

In [7]:
user_id = user_df['phone_number'].to_list()

In [8]:
item_id = item_df['pushcall_id'].to_list()

In [9]:
item_dic = [{'id':x,'relevancy':0} for x in item_id]

In [10]:
int(datetime.now().timestamp()*1000)

1667664945249

In [11]:
str(uuid.uuid4())

'3887b41d-3743-4a6d-86e0-78d922cbd0da'

In [12]:
all_requests = [{
    'user':str(u_id),
    'timestamp':int(datetime.now().timestamp()*1000),
    'items':item_dic,
    'id':str(uuid.uuid4()),
    'event':'ranking',
    'session':u_id,
    'tenant': 'default', 'fields': []} for u_id in user_id]

In [13]:
assert len(all_requests)==len(user_id),"All user weren't created"
assert len(all_requests[0]['items'])==len(item_id),"All items weren't added"

In [25]:
url_rankings_explain = "http://localhost:8080/rank/xgboost?explain=TRUE"
url_rankings = "http://localhost:8080/rank/xgboost"
url_feedback = "http://localhost:8080/feedback"

In [26]:
headers = {
  'Content-Type': 'application/json'
}

In [ ]:
# this will store the vector for the user
user_ranking_dic = {}
user_item_ranking_dic = {}

for req in all_requests:
    usr = req['user']
    payload = json.dumps(req)
    response = requests.request("POST", url_rankings, headers=headers, data=payload,timeout=10)
    json1_data = json.loads(response.text)
    ranking = json1_data['items']
    # this will store the rank for each tag (tag here means base_tag|tag_1|crop)
    item_tag_rank = {}
    for item_rank in ranking:
        item_id = item_rank['item']
        user_item_ranking_dic.setdefault(item_id,[])
        base_id = idx_to_base_tag2_crop_map[item_id]
        score = item_rank['score']
        user_item_ranking_dic[item_id].append(score)
        item_tag_rank.setdefault(base_id,[])
        item_tag_rank[base_id].append(score)
        
    # get average of score for each tag and push it in user_ranking_dic
    user_ranking_dic.setdefault('user',[])
    user_item_ranking_dic.setdefault('user',[])
    user_ranking_dic['user'].append(usr)
    user_item_ranking_dic['user'].append(usr)
    for k,v in item_tag_rank.items():
        user_ranking_dic.setdefault(k,[])
        avg_item_score = sum(v)/len(v)
        user_ranking_dic[k].append(avg_item_score)

ranking_dic = pd.DataFrame(user_item_ranking_dic)
df = pd.DataFrame(user_ranking_dic)
df.to_csv('./user_item_vec.csv',index=False)
ranking_dic.to_csv('./user_item_ranking.csv',index=False)